# Azure AI Agent ServiceとSemantic Kernelを使用したRAG

このコードスニペットは、`Azure AI Agent Service`と`Semantic Kernel`を使用して、情報検索強化生成（RAG）のためのAzure AIエージェントを作成および管理する方法を示しています。エージェントは取得したコンテキストに基づいてユーザーのクエリを処理し、それに応じて正確な回答を提供します。


SQLite バージョン修正  
以下のエラーが発生した場合:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

ノートブックの冒頭でこのコードブロックのコメントを解除してください:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### パッケージのインポート
以下のコードは必要なパッケージをインポートします:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# セマンティックカーネルとAzure AI Agent Serviceを用いた検索強化型生成

このサンプルでは、**Azure AI Agent Service** を使用して、アップロードされたドキュメントから得られるドメイン固有のコンテキストと言語モデルを組み合わせることで、**検索強化型生成 (RAG)** を実現する方法を示します。

### 動作の仕組み

1. **ドキュメントのアップロード**: 情報（Contosoの旅行保険ポリシー）を含むMarkdownファイル (document.md) をエージェントサービスにアップロードします。

2. **ベクトルストアの作成**: ドキュメントをインデックス化し、その内容に対するセマンティック検索を可能にするベクトルストアを作成します。

3. **エージェントの設定**: 以下の厳密な指示を伴う `gpt-4o` モデルを使用してエージェントを構築します:
   - ドキュメントから取得した内容に基づいてのみ質問に回答する。
   - 質問が範囲外の場合は回答を拒否する。

4. **ファイル検索ツールの統合**: エージェントに `FileSearchTool` を登録し、推論中にインデックス化されたドキュメントから関連するスニペットを検索・取得できるようにします。

5. **ユーザーとの対話**: ユーザーは質問をすることができます。ドキュメント内に関連情報が見つかった場合、エージェントは根拠のある回答を生成します。  
   見つからない場合、エージェントはドキュメントに十分な情報が含まれていないことを明確に回答します。


### メイン関数

```python
def main():
    # プログラムのエントリーポイント
    print("Hello, World!")
```

### 概要

このドキュメントでは、Pythonでの基本的なメイン関数の構造について説明します。

### 使用方法

1. Pythonがインストールされていることを確認してください。
2. ターミナルで以下のコマンドを実行します:

   ```bash
   python main.py
   ```

3. "Hello, World!" が出力されることを確認してください。

### 注意事項

- このコードは教育目的で提供されています。
- 実際のプロジェクトでは、より複雑なエラーハンドリングやロギングが必要になる場合があります。

### 次のステップ

- メイン関数に引数を追加して、動的な動作を実現する方法を学びましょう。
- Pythonの標準ライブラリを活用して、より高度な機能を実装してみてください。


Azure CLI を使用して適切な認証コンテキストが提供されるように、最初に `az login` を実行してください。その後、`DefaultAzureCredential` を使用します。Azure AI Agent Service は API キーを使用しません。


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

了解しました。翻訳を始めます。

---

# プロジェクト概要

このプロジェクトは、@@INLINE_CODE_0@@を使用してデータを処理し、@@INLINE_CODE_1@@を生成することを目的としています。以下の手順に従ってください。

## 必要条件

- Python 3.8以上
- 必須ライブラリ: @@INLINE_CODE_2@@, @@INLINE_CODE_3@@

## インストール方法

1. リポジトリをクローンします:
   ```bash
   ```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```
   ```
2. 必要な依存関係をインストールします:
   ```bash
   @@CODE_BLOCK_1@@
   ```

## 使用方法

1. スクリプトを実行する前に、@@INLINE_CODE_4@@ファイルを設定してください。
2. 以下のコマンドを使用してスクリプトを実行します:
   ```bash
   @@CODE_BLOCK_2@@
   ```

[!NOTE] 結果は@@INLINE_CODE_5@@ディレクトリに保存されます。

## 注意事項

[!WARNING] データが正しくフォーマットされていない場合、エラーが発生する可能性があります。

[!TIP] サンプルデータは@@INLINE_CODE_6@@ディレクトリにあります。

## 貢献方法

1. フォークしてください。
2. 新しいブランチを作成します:
   ```bash
   @@CODE_BLOCK_3@@
   ```
3. 変更をコミットします:
   ```bash
   @@CODE_BLOCK_4@@
   ```
4. プルリクエストを送信してください。

## ライセンス

このプロジェクトは@@INLINE_CODE_7@@ライセンスの下で提供されています。

---

翻訳が完了しました。



---

**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤った解釈について、当方は責任を負いません。
